In [24]:
import numpy as np
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
import pandas as pd
from pandas.api.types import is_integer_dtype, is_float_dtype

load_dotenv()
os.chdir(os.getenv('BASE_PATH'))


In [25]:
df = pd.read_csv('data/raw/retail_store_inventory.csv')
df.drop(columns=['Date', 'Store ID', 'Product ID'], axis=1, inplace=True)

In [26]:
df = df.rename(columns={'Inventory Level': 'Inventory',
                        'Units Sold': 'Sales',
                        'Units Ordered': 'Orders',
                        'Demand Forecast': 'Demand',
                        'Weather Condition': 'Weather',
                        'Holiday/Promotion': 'Promotion',
                        'Competitor Pricing': 'Competitor Price'})

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Category          73100 non-null  object 
 1   Region            73100 non-null  object 
 2   Inventory         73100 non-null  int64  
 3   Sales             73100 non-null  int64  
 4   Orders            73100 non-null  int64  
 5   Demand            73100 non-null  float64
 6   Price             73100 non-null  float64
 7   Discount          73100 non-null  int64  
 8   Weather           73100 non-null  object 
 9   Promotion         73100 non-null  int64  
 10  Competitor Price  73100 non-null  float64
 11  Seasonality       73100 non-null  object 
dtypes: float64(3), int64(5), object(4)
memory usage: 6.7+ MB


In [28]:
def downcast_ints(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    int_cols = [c for c in df.columns if is_integer_dtype(df[c])]
    for c in int_cols:
        s = df[c]
        if s.min() >= 0:
            df[c] = pd.to_numeric(s, downcast="unsigned")  # -> uint8/16/32/64
        else:
            df[c] = pd.to_numeric(s, downcast="integer")   # -> int8/16/32/64
    return df

def optimize_df(df: pd.DataFrame) -> pd.DataFrame:
    start = df.memory_usage(deep=True).sum()
    out = df.copy()

    # 1) Downcast integers
    out = downcast_ints(out)

    # 2) Downcast floats
    float_cols = [c for c in out.columns if is_float_dtype(out[c])]
    out[float_cols] = out[float_cols].apply(pd.to_numeric, downcast="float")  # -> float32

    end = out.memory_usage(deep=True).sum()
    print(f"Memory: {start/1024**2:.2f} MB → {end/1024**2:.2f} MB "
          f"({(1 - end/start)*100:.1f}% reduction)")
    return out

df = optimize_df(df)
df.info()


Memory: 22.03 MB → 18.89 MB (14.2% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Category          73100 non-null  object 
 1   Region            73100 non-null  object 
 2   Inventory         73100 non-null  uint16 
 3   Sales             73100 non-null  uint16 
 4   Orders            73100 non-null  uint8  
 5   Demand            73100 non-null  float32
 6   Price             73100 non-null  float32
 7   Discount          73100 non-null  uint8  
 8   Weather           73100 non-null  object 
 9   Promotion         73100 non-null  uint8  
 10  Competitor Price  73100 non-null  float32
 11  Seasonality       73100 non-null  object 
dtypes: float32(3), object(4), uint16(2), uint8(3)
memory usage: 3.6+ MB


In [29]:
x = df.drop(columns=['Demand'])
y = df['Demand']

In [30]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
x['Category'] = label_encoder.fit_transform(x['Category'])
x['Region'] = label_encoder.fit_transform(x['Region'])
x['Weather'] = label_encoder.fit_transform(x['Weather'])
x['Seasonality'] = label_encoder.fit_transform(x['Seasonality'])
x['Promotion'] = label_encoder.fit_transform(x['Promotion'])

In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [32]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [33]:
class HParamMetrics:

  def __init__(self, est):
    self.est = est
    self.train_rn = None
    self.y_pred = None
    self.test_r2 = None
    self.mse = None
    self.rmse = None
    self.mae = None

In [34]:
from tabulate import tabulate
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate_model(model, x_train, y_train, x_test, y_test, HPM):
  # calculate metrics
  HPM.train_r2 = model.score(x_train, y_train)
  HPM.y_pred = model.predict(x_test)
  HPM.test_r2 = r2_score(y_test, HPM.y_pred)
  HPM.mse = mean_squared_error(y_test, HPM.y_pred)
  HPM.rmse = np.sqrt(HPM.mse)
  HPM.mae = mean_absolute_error(y_test, HPM.y_pred)

  #prepare the table data
  metrics = [
    ["Training R^2 Score", f"{HPM.train_r2:.4f}"],
    ["Testing R^2 Score", f"{HPM.test_r2:.4f}"],
    ["Mean Squared Error (MSE)", f"{HPM.mse:.2f}"],
    ["Root Mean Squared Error (RMSE)", f"{HPM.rmse:.2f}"],
    ["Mean Absolute Error (MAE)", f"{HPM.mae:.2f}"],
  ]

  # Print the table
  print(tabulate(metrics, headers=["Metric", "Value"], tablefmt="grid"))

In [35]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

initial_HPM = HParamMetrics(lr.fit(x_train, y_train))

In [36]:
evaluate_model(lr, x_train, y_train, x_test, y_test, initial_HPM)

+--------------------------------+---------+
| Metric                         |   Value |
+================================+=========+
| Training R^2 Score             |  0.9937 |
+--------------------------------+---------+
| Testing R^2 Score              |  0.9936 |
+--------------------------------+---------+
| Mean Squared Error (MSE)       | 75.34   |
+--------------------------------+---------+
| Root Mean Squared Error (RMSE) |  8.68   |
+--------------------------------+---------+
| Mean Absolute Error (MAE)      |  7.51   |
+--------------------------------+---------+


In [37]:
print(initial_HPM.est)

LinearRegression()


In [41]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {'copy_X': [True,False],
               'fit_intercept': [True,False],
               'n_jobs': [1,5,10,15,None],
               'positive': [True,False]}


rs = RandomizedSearchCV(
	initial_HPM.est,
	param_distributions=param_dist,
	n_iter=10,
	verbose=1,
  cv=5,
)

rs.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,estimator,LinearRegression()
,param_distributions,"{'copy_X': [True, False], 'fit_intercept': [True, False], 'n_jobs': [1, 5, ...], 'positive': [True, False]}"
,n_iter,10
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [42]:
rs.cv_results_

{'mean_fit_time': array([0.04987688, 0.03830252, 0.01545677, 0.03076243, 0.03291202,
        0.01347351, 0.02702341, 0.01510267, 0.01527972, 0.01552005]),
 'std_fit_time': array([0.0092254 , 0.00272485, 0.00284253, 0.00603891, 0.00192778,
        0.00064111, 0.00240799, 0.00084464, 0.00122747, 0.0005612 ]),
 'mean_score_time': array([0.00185666, 0.00175438, 0.00145712, 0.00179291, 0.00204053,
        0.0019074 , 0.00219841, 0.00227399, 0.00237627, 0.00263104]),
 'std_score_time': array([0.00019257, 0.00036952, 0.00028797, 0.00053093, 0.00046266,
        0.00021402, 0.00041047, 0.00051553, 0.00071658, 0.00051418]),
 'param_positive': masked_array(data=[True, True, False, True, True, False, True, False,
                    False, False],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=True),
 'param_n_jobs': masked_array(data=[15, 1, 15, 10, None, 1, 1, 10, 1, None],
              mask=[False, False, False,